In [1]:
import fastf1 as ff1
from fastf1 import plotting
from fastf1.core import Laps

import pandas as pd

In [56]:
rf_22 = pd.read_csv('features_race_2022.csv')
rf_23 = pd.read_csv('features_race_2023.csv')
rf_24 = pd.read_csv('features_race_2024.csv')
rf_25 = pd.read_csv('features_race_2025.csv')

rf = [rf_22, rf_23, rf_24, rf_25]

rf = pd.concat(rf, ignore_index=True)
rf.to_csv('features_race_p3.csv', index=False)

In [57]:
rf = pd.read_csv('features_race_p3.csv', index_col=False)

In [ ]:
import pandas as pd
import numpy as np

# --- ASSUMPTION: Load your fully consolidated Master Race Pace Data ---
# master_race_pace_df = pd.read_csv('your_master_race_pace_data.csv')

# --- 1. Calculate LapTime_Relative (Required for a meaningful pace score) ---
# First, calculate the mean lap time for *each race event* to normalize
location_year_mean = master_race_pace_df.groupby(['Location', 'Year'])['LapTime'].mean().rename('LapTime_Race_Mean')
master_race_pace_df = master_race_pace_df.merge(location_year_mean, on=['Location', 'Year'])

# Calculate the Relative Lap Time (A negative value is faster)
master_race_pace_df['LapTime_Relative'] = master_race_pace_df['LapTime'] - master_race_pace_df['LapTime_Race_Mean']


# --- 2. Define Aggregation Functions for Driver Score ---
driver_scores = master_race_pace_df.groupby(['Driver', 'Year']).agg(
    # The primary performance score: Mean of relative lap times.
    # A negative value means the driver was, on average, faster than the field.
    Driver_Relative_Pace=('LapTime_Relative', 'mean'), 
    
    # Consistency Index (Idea: Standard deviation of relative pace)
    # Lower value means the driver is more consistent
    Driver_Consistency_Index=('LapTime_Relative', 'std'),
    
    # Count of valid laps driven (a measure of data quality/completeness)
    Total_Valid_Laps=('LapTime', 'count'), 
    
    # Average environmental conditions experienced (useful for context, not a score)
    Avg_TrackTemp=('TrackTemp', 'mean'),
    
    # Keep the latest Team for the merge key (Crucial for Teammate Gap calculation later)
    Current_Team=('Team', lambda x: x.iloc[-1]) 
    
).reset_index()


# --- 3. Preview the Result ---
# This dataframe now provides one row per Driver per Year, 
# summarizing their average performance and consistency across the season.

print("--- Driver Seasonal Performance Scores Head ---")
print(driver_scores.head())

# Save this for later merging into your qualifying data
# driver_scores.to_csv('driver_seasonal_scores.csv', index=False)

In [58]:
rf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74026 entries, 0 to 74025
Data columns (total 38 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             74026 non-null  object 
 1   Driver           74026 non-null  object 
 2   DriverNumber     74026 non-null  int64  
 3   Team             74026 non-null  object 
 4   LapTime          74026 non-null  float64
 5   LapNumber        74026 non-null  float64
 6   Stint            74026 non-null  float64
 7   Sector1Time      74026 non-null  float64
 8   Sector2Time      74026 non-null  float64
 9   Sector3Time      74026 non-null  float64
 10  Compound         74026 non-null  object 
 11  TyreLife         74026 non-null  float64
 12  FreshTyre        74026 non-null  bool   
 13  Position         74026 non-null  float64
 14  AirTemp          74026 non-null  float64
 15  TrackTemp        74026 non-null  float64
 16  Rainfall         74026 non-null  bool   
 17  Humidity    

In [22]:
# Enable the cache by providing the name of the cache folder, speed up
ff1.Cache.enable_cache('cache')

# Setup plotting, setup the plot (bg: black, ...)
plotting.setup_mpl()

c:\Users\lucam\anaconda3\envs\myenv_f1\lib\site-packages\fastf1\plotting\_plotting.py:90: FutureWarning: FastF1 will no longer silently modify the default Matplotlib colors in the future.
To remove this warning, explicitly set `color_scheme=None` or `color_scheme='fastf1'` when calling `.setup_mpl()`.
  warnings.warn(
c:\Users\lucam\anaconda3\envs\myenv_f1\lib\site-packages\fastf1\plotting\_plotting.py:98: FutureWarning: FastF1 will stop modifying the default Matplotlib settings in the future.
To opt-in to the new behaviour and remove this warning, explicitly set `misc_mpl_mods=False` when calling `.setup_mpl()`.
  warnings.warn(


In [21]:
ff1.Cache.offline_mode(True)

req         WARNING 	DEFAULT CACHE ENABLED! (114.74 KB) C:\Users\lucam\AppData\Local\Temp\fastf1


In [2]:
FILE_CIRCUITS = 'f1_unique_circuits_complete.csv'

circuit_info = pd.read_csv(FILE_CIRCUITS, delimiter=",")

In [6]:
circuit_info['Location'][23]

'Yas Island'

In [ ]:
"""year = 2022

ff1.set_log_level('WARNING')

for location in circuit_info['Location'][23]:
    try:
        print(location + "    " + str(year))
        session = ff1.get_session(year, location, 'R')
        session.load()
    except Exception as e:
        print(f"Skipped {location} {year}: {e}")"""

Y    2022


events      WARNING 	Correcting user input 'Y' to 'Hungarian Grand Prix'
events      WARNING 	Correcting user input 'a' to 'Azerbaijan Grand Prix'


a    2022


core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
events      WARNING 	Correcting user input 's' to 'Spanish Grand Prix'


s    2022


...

core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
events      WARNING 	Correcting user input 'n' to 'Spanish Grand Prix'


n    2022


events      WARNING 	Correcting user input 'd' to 'Dutch Grand Prix'


d    2022


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001BB03D0B5B0>>
Traceback (most recent call last):
  File "c:\Users\lucam\anaconda3\envs\myenv_f1\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [53]:
race_features_sorted.to_csv('miami.csv')

In [ ]:
YEAR = 2025
locations = ff1.get_event_schedule(YEAR)['Location'][1:20]

"""
import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)
"""

all_races = []

for LOCATION in locations:
    try:
        session = ff1.get_session(YEAR, LOCATION, 'R')
        session.load(laps=True, telemetry=True, weather=True)
        laps = session.laps

        laps_filtered = laps.copy()
        # 1. Remove laps marked as inaccurate or where LapTime is missing
        laps_filtered = laps_filtered.loc[
            (laps_filtered['IsAccurate'] == True) &
            (laps_filtered['LapTime'].notnull())
        ]

        # 2. Remove in-laps and out-laps (around a pit stop)
        laps_filtered = laps_filtered.loc[
            (laps_filtered['PitInTime'].isnull()) &
            (laps_filtered['PitOutTime'].isnull())
        ]

        # 3. Remove laps run under Safety Car, VSC, or Red Flag
        # '1' means Green Flag / Clear Track
        laps_filtered = laps_filtered.loc[laps_filtered['TrackStatus'] == '1']

        laps_filtered = laps_filtered.copy()
        laps_filtered['Location'] = LOCATION
        laps_filtered['Year'] = YEAR

        # Filter laps that are no more than 108% of the fastest lap time
        # This is a common tolerance used in racing for pace comparison
        # laps_filtered = laps_filtered.loc[laps_filtered['LapTime'] < fastest_lap_time * 1.08]

        weather = session.weather_data
        laps_with_weather = pd.merge_asof(
            laps_filtered.sort_values('LapStartTime'),
            weather.sort_values('Time'),
            left_on='LapStartTime',
            right_on='Time',
            direction = 'backward'    # use last known weather measurement
        )

        laps_with_weather = laps_with_weather.rename(columns={'Time_x': 'Time'})
        laps_with_weather = laps_with_weather.rename(columns={'Time_y': 'WeatherTime'})

        # merge with circuit info
        laps_with_weather_circuits = pd.merge(laps_with_weather, circuit_info, on='Location', how='left')

        FINAL_COLS = [
            'Time', 'Driver', 'DriverNumber', 'Team', 'LapTime', 'LapNumber', 'Stint',
            'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 'TyreLife', 'FreshTyre', 'Position',
            'TyreLife', 'AirTemp', 'TrackTemp', 'Rainfall', 'Humidity', 'WindSpeed', 'WindDirection', 
            'TrackLenght', 'NumTurns','NumLaps', 'RaceDistance', 'Traction', 'AsphaltGrip', 
            'AsphaltAbrasion', 'TrackEvolution', 'TyreStress', 'Braking', 'LateralLoad','Downforce', 
            'Soft', 'Medium', 'Hard', 'Location', 'Country', 'Year'
        ]

        race_features = laps_with_weather_circuits[FINAL_COLS]

        race_features['LapTime'] = race_features['LapTime'].dt.total_seconds()
        race_features['Sector1Time'] = race_features['Sector1Time'].dt.total_seconds()
        race_features['Sector2Time'] = race_features['Sector2Time'].dt.total_seconds()
        race_features['Sector3Time'] = race_features['Sector3Time'].dt.total_seconds()


        race_features_sorted = (
            race_features
            .sort_values(['Driver', 'Time'])
            .groupby('Driver', group_keys=True)
            .apply(lambda x: x)
        )
        
        all_races.append(race_features_sorted)
    except Exception as e:
            print(f"Skipped {LOCATION} {YEAR}: {e}")

all_features_df = pd.concat(all_races, ignore_index=True)

filename = 'features_race_' + str(YEAR) + '.csv'
all_features_df.to_csv(filename, index=False)


In [23]:
race_features[['Time', 'Driver']]

,Time,Driver
0,0 days 01:29:42.812000,NOR
1,0 days 01:29:43.766000,VER
2,0 days 01:29:44.822000,PIA
3,0 days 01:29:46.011000,RUS
4,0 days 01:29:47.114000,LEC
...,...,...
546,0 days 02:53:27.318000,PIA
547,0 days 02:53:33.398000,GAS
548,0 days 02:53:36.816000,TSU
549,0 days 02:53:40.059000,OCO


In [15]:
laps_with_weather_circuits.info()

<class 'fastf1.core.Laps'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 59 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Time                551 non-null    timedelta64[ns]
 1   Driver              551 non-null    object         
 2   DriverNumber        551 non-null    object         
 3   LapTime             551 non-null    timedelta64[ns]
 4   LapNumber           551 non-null    float64        
 5   Stint               551 non-null    float64        
 6   PitOutTime          0 non-null      timedelta64[ns]
 7   PitInTime           0 non-null      timedelta64[ns]
 8   Sector1Time         551 non-null    timedelta64[ns]
 9   Sector2Time         551 non-null    timedelta64[ns]
 10  Sector3Time         551 non-null    timedelta64[ns]
 11  Sector1SessionTime  551 non-null    timedelta64[ns]
 12  Sector2SessionTime  551 non-null    timedelta64[ns]
 13  Sector3SessionTime  551 non-null    timedelta6

In [120]:
laps_with_weather.groupby(['Driver', 'Location'])

In [81]:
laps.get(635)

In [79]:
laps.pick_fastest()

Time                      0 days 02:28:03.586000
Driver                                       NOR
DriverNumber                                   4
LapTime                   0 days 00:01:22.167000
LapNumber                                   43.0
Stint                                        5.0
PitOutTime                                   NaT
PitInTime                                    NaT
Sector1Time               0 days 00:00:28.553000
Sector2Time               0 days 00:00:18.537000
Sector3Time               0 days 00:00:35.077000
Sector1SessionTime        0 days 02:27:10.030000
Sector2SessionTime        0 days 02:27:28.567000
Sector3SessionTime        0 days 02:28:03.644000
SpeedI1                                    271.0
SpeedI2                                    290.0
SpeedFL                                    292.0
SpeedST                                    300.0
IsPersonalBest                              True
Compound                                    HARD
TyreLife            

In [108]:
laps[laps['Driver'] == 'VER']

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:13:00.002000,VER,1,0 days 00:01:59.392000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:20.705000,...,True,Red Bull Racing,0 days 01:11:00.355000,2025-03-16 04:18:22.974,124,5.0,False,,False,False
1,0 days 01:15:49.358000,VER,1,NaT,2.0,1.0,NaT,0 days 01:15:38.205000,0 days 00:00:58.141000,0 days 00:00:37.976000,...,True,Red Bull Racing,0 days 01:13:00.002000,2025-03-16 04:20:22.621,4,2.0,False,,False,False
2,0 days 01:18:31.526000,VER,1,NaT,3.0,2.0,0 days 01:15:51.658000,0 days 01:18:20.223000,0 days 00:00:56.230000,0 days 00:00:33.683000,...,False,Red Bull Racing,0 days 01:15:49.358000,2025-03-16 04:23:11.977,4,2.0,False,,False,False
3,0 days 01:21:07.226000,VER,1,NaT,4.0,3.0,0 days 01:18:34.029000,0 days 01:20:56.543000,0 days 00:00:54.351000,0 days 00:00:32.712000,...,False,Red Bull Racing,0 days 01:18:31.526000,2025-03-16 04:25:54.145,4,2.0,False,,False,False
4,0 days 01:23:30.835000,VER,1,0 days 00:02:23.609000,5.0,4.0,0 days 01:21:09.534000,NaT,0 days 00:00:53.513000,0 days 00:00:32.627000,...,False,Red Bull Racing,0 days 01:21:07.226000,2025-03-16 04:28:29.845,4,2.0,False,,False,False
5,0 days 01:25:50.489000,VER,1,0 days 00:02:19.654000,6.0,4.0,NaT,NaT,0 days 00:00:47.887000,0 days 00:00:33.228000,...,False,Red Bull Racing,0 days 01:23:30.835000,2025-03-16 04:30:53.454,4,2.0,False,,False,False
6,0 days 01:28:06.936000,VER,1,0 days 00:02:16.447000,7.0,4.0,NaT,NaT,0 days 00:00:45.898000,0 days 00:00:29.735000,...,False,Red Bull Racing,0 days 01:25:50.489000,2025-03-16 04:33:13.108,41,2.0,False,,False,False
7,0 days 01:29:43.766000,VER,1,0 days 00:01:36.830000,8.0,4.0,NaT,NaT,0 days 00:00:35.249000,0 days 00:00:20.145000,...,False,Red Bull Racing,0 days 01:28:06.936000,2025-03-16 04:35:29.555,1,2.0,False,,False,True
8,0 days 01:31:18.081000,VER,1,0 days 00:01:34.315000,9.0,4.0,NaT,NaT,0 days 00:00:33.850000,0 days 00:00:19.608000,...,False,Red Bull Racing,0 days 01:29:43.766000,2025-03-16 04:37:06.385,1,2.0,False,,False,True
9,0 days 01:32:50.731000,VER,1,0 days 00:01:32.650000,10.0,4.0,NaT,NaT,0 days 00:00:33.117000,0 days 00:00:19.414000,...,False,Red Bull Racing,0 days 01:31:18.081000,2025-03-16 04:38:40.700,1,2.0,False,,False,True


In [30]:
session = ff1.get_session(2022, 'Barcelona', 'R')

In [31]:
session.load()

In [37]:
ff1.get_event_schedule(2023)['Location']

0                Sakhir
1                Sakhir
2                Jeddah
3             Melbourne
4                  Baku
5                 Miami
6                Monaco
7             Barcelona
8              Montréal
9             Spielberg
10          Silverstone
11             Budapest
12    Spa-Francorchamps
13            Zandvoort
14                Monza
15           Marina Bay
16               Suzuka
17               Lusail
18               Austin
19          Mexico City
20            São Paulo
21            Las Vegas
22           Yas Island
Name: Location, dtype: object

In [46]:
features_race = pd.read_csv('features_race_2025.csv')

In [47]:
features_race.isna().sum()

Time               0
Driver             0
DriverNumber       0
Team               0
LapTime            0
LapNumber          0
Stint              0
Sector1Time        0
Sector2Time        0
Sector3Time        0
Compound           0
TyreLife           0
FreshTyre          0
Position           0
AirTemp            0
TrackTemp          0
Rainfall           0
Humidity           0
WindSpeed          0
WindDirection      0
TrackLenght        0
NumTurns           0
NumLaps            0
RaceDistance       0
Traction           0
AsphaltGrip        0
AsphaltAbrasion    0
TrackEvolution     0
TyreStress         0
Braking            0
LateralLoad        0
Downforce          0
Soft               0
Medium             0
Hard               0
Location           0
Country            0
Year               0
dtype: int64

In [29]:
df = pd.read_csv('features_race_2025.csv')

In [30]:
df.isna().sum()

Unnamed: 0           0
Time                 0
Driver               0
DriverNumber         0
Team                 0
LapTime              0
LapNumber            0
Stint                0
Sector1Time          0
Sector2Time          0
Sector3Time          0
Compound             0
TyreLife             0
FreshTyre            0
Position             0
TyreLife.1         382
AirTemp              0
TrackTemp            0
Rainfall             0
Humidity             0
WindSpeed            0
WindDirection        0
TrackLenght          0
NumTurns             0
NumLaps              0
RaceDistance         0
Traction             0
AsphaltGrip          0
AsphaltAbrasion      0
TrackEvolution       0
TyreStress           0
Braking              0
LateralLoad          0
Downforce            0
Soft                 0
Medium               0
Hard                 0
Location             0
Country              0
Year                 0
dtype: int64

In [ ]:
df['Stint'] = df['Stint'].fillna(1.0)

# Group by both Driver and Stint
# .bfill() fills gaps from the next valid row *in that stint*
# .ffill() fills gaps from the previous valid row *in that stint*
df['Compound'] = df.groupby(['Driver', 'Stint'])['Compound'].bfill().ffill()

df.loc[df['Stint'] == 1.0, 'TyreLife'] = df['LapNumber']

#df.to_csv('features_race_2025.csv')